<a href="https://colab.research.google.com/github/JoseDanielRojas/Analisis-de-algoritmos/blob/master/Proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto 3:** Transfer Learning

## Inicialización del Dataset.

### Instalación e importación de librerias.

In [ ]:
!pip install hydra-core --upgrade
!pip install wandb
!pip install torch torchvision torchaudio pytorch-lightning

In [ ]:
import sys
import pandas as pd
import numpy as np
import hydra
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra
from omegaconf import DictConfig
import pytorch_lightning as pl
from pytorch_lightning import LightningModule
from pytorch_lightning import LightningDataModule
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
import torchvision
from torchvision import transforms, datasets
import wandb
import os
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

### Montaje y acceso al dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
rutaDataset = '/content/drive/MyDrive/ButterfliesDataset_filtrado'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Inicialización de wandb.

In [ ]:
wandb.login(relogin=True)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Inicialización Hydra.

In [ ]:
config_yaml = """
defaults:
  - override hydra/job_logging: disabled

experiment:
  data_dir: /content/drive/MyDrive/ButterfliesDataset_filtrado
  batch_size: 32
  labeled_ratio: 0.3
  max_epochs_autoencoder: 75
  max_epochs_classifier_a: 50
  max_epochs_classifier_b1: 30
  max_epochs_classifier_b2: 50
  learning_rate_autoencoder: 1e-3
  learning_rate_classifier_a: 1e-4
  learning_rate_classifier_b1: 1e-3
  learning_rate_classifier_b2: 1e-4
"""

# Guardar la configuración YAML en un archivo temporal
with open("config.yaml", "w") as file:
    file.write(config_yaml)

### Transformación para redimensión de imagenes.

In [ ]:
# Data transformations
transformacionOriginal = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

transformacionRedimensionada = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### Selección de 20 especias con mayor cantidad de muestras.

In [ ]:
# Seleccionar las 20 especies con mayor cantidad de muestras y mover muestras al set de testing
datosEspecies = datasets.ImageFolder(root=os.path.join(rutaDataset, 'train'), transform=transformacionOriginal)
conteoEspecies = Counter([label for _, label in datosEspecies.imgs])

# Obtener las 20 especies con mayor cantidad de muestras
especiesMasComunes = conteoEspecies.most_common(20)
idsEspeciesMasComunes = [especie[0] for especie in especiesMasComunes]

In [ ]:
# Crear nuevos directorios para los sets de entrenamiento y prueba filtrados
directorioEntrenamientoFiltrado = '/content/drive/MyDrive/ButterfliesDataset_filtrado/train'
directorioPruebaFiltrado = '/content/drive/MyDrive/ButterfliesDataset_filtrado/test'

os.makedirs(directorioEntrenamientoFiltrado, exist_ok=True)
os.makedirs(directorioPruebaFiltrado, exist_ok=True)

### Filtrado de imagenes y movimiento de muestras.

    # for idEspecie in idsEspeciesMasComunes:
    # Obtener todas las imágenes correspondientes a la especie actual
    imagenesEspecie = [img for img, label in datosEspecies.imgs if label == idEspecie]
    # Dividir las imágenes en 20 para prueba y el resto para entrenamiento
    imagenesPrueba = imagenesEspecie[:20]
    imagenesEntrenamiento = imagenesEspecie[20:]

    # Crear carpetas para cada especie dentro de los directorios de entrenamiento y prueba
    nombreEspecie = datosEspecies.classes[idEspecie]
    os.makedirs(os.path.join(directorioEntrenamientoFiltrado, nombreEspecie), exist_ok=True)
    os.makedirs(os.path.join(directorioPruebaFiltrado, nombreEspecie), exist_ok=True)

    # Mover las imágenes de entrenamiento al directorio de entrenamiento
    for rutaImagen in imagenesEntrenamiento:
        # Cargar la imagen desde la ruta
        imagen = datasets.folder.default_loader(rutaImagen)
        # Redimensionar la imagen a 128x128 usando la transformación definida
        imagenTransformada = transformacionRedimensionada(imagen)
        # Definir la ruta de destino para guardar la imagen redimensionada
        rutaDestino = os.path.join(directorioEntrenamientoFiltrado, nombreEspecie, os.path.basename(rutaImagen))
        # Guardar la imagen transformada en el directorio de entrenamiento
        torchvision.utils.save_image(imagenTransformada, rutaDestino)

    # Mover las imágenes de prueba al directorio de prueba
    for rutaImagen in imagenesPrueba:
        # Cargar la imagen desde la ruta
        imagen = datasets.folder.default_loader(rutaImagen)
        # Redimensionar la imagen a 128x128 usando la transformación definida
        imagenTransformada = transformacionRedimensionada(imagen)
        # Definir la ruta de destino para guardar la imagen redimensionada
        rutaDestino = os.path.join(directorioPruebaFiltrado, nombreEspecie, os.path.basename(rutaImagen))
        # Guardar la imagen transformada en el directorio de prueba
        torchvision.utils.save_image(imagenTransformada, rutaDestino)

    print("Las muestras se han movido correctamente a los sets de entrenamiento y prueba filtrados con redimensionamiento.")

## Experimento 1.

### Inicialización.

#### Creación del LightningDataModule.

In [ ]:
class ButterfliesDataModule(LightningDataModule):
    def __init__(self, data_dir, batch_size, labeled_ratio):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.labeled_ratio = labeled_ratio
        self.transform = transformacionRedimensionada

    def setup(self, stage=None):
        # Cargar el dataset completo utilizando la transformación definida
        dataset = datasets.ImageFolder(root=self.data_dir, transform=self.transform)
        total_size = len(dataset)

        # Definir el tamaño del conjunto con labels y el conjunto sin labels
        labeled_size = int(self.labeled_ratio * total_size)
        unlabeled_size = total_size - labeled_size

        # Dividir el dataset en los conjuntos etiquetados y no etiquetados
        self.labeled_data, self.unlabeled_data = random_split(dataset, [labeled_size, unlabeled_size])

    def train_dataloader(self):
        # Retornar un DataLoader para el conjunto de entrenamiento del autoencoder (datos sin etiquetas)
        return DataLoader(self.unlabeled_data, batch_size=self.batch_size, shuffle=True)

    def labeled_train_dataloader(self):
        # Retornar un DataLoader para el conjunto de entrenamiento de los clasificadores (datos con etiquetas)
        return DataLoader(self.labeled_data, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        # Retornar un DataLoader para el conjunto de validación (datos con etiquetas)
        return DataLoader(self.labeled_data, batch_size=self.batch_size)

    def test_dataloader(self):
        # Retornar un DataLoader para el conjunto de prueba (datos con etiquetas)
        return DataLoader(self.labeled_data, batch_size=self.batch_size)


#### Creación del Autoencoder basado en U-net.

In [ ]:
class UNetAutoencoder(LightningModule):
    def __init__(self, learning_rate=1e-3):
        super().__init__()
        self.learning_rate = learning_rate

        # Encoder
        # Primera parte del encoder: Conv2d -> BatchNorm -> ReLU
        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Conv con 3 canales de entrada (RGB), 64 canales de salida
            nn.BatchNorm2d(64),  # Normalización por lotes para mejorar la estabilidad del entrenamiento
            nn.ReLU()  # Activación ReLU
        )
        self.pool1 = nn.MaxPool2d(2, 2)  # Operación de max-pooling para reducir la resolución espacial a la mitad

        # Segunda parte del encoder: Conv2d -> BatchNorm -> ReLU
        self.encoder2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Conv con 64 canales de entrada, 128 canales de salida
            nn.BatchNorm2d(128),  # Normalización por lotes
            nn.ReLU()  # Activación ReLU
        )
        self.pool2 = nn.MaxPool2d(2, 2)  # Max-pooling para reducir aún más la resolución espacial

        # Parte del Bottleneck (representación latente)
        self.bottleneck = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Conv para comprimir las características a 256 canales
            nn.BatchNorm2d(256),  # Normalización por lotes
            nn.ReLU()  # Activación ReLU
        )

        # Decoder con Skip Connections
        # Up-sampling de la resolución espacial: de 256 a 128 canales
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = nn.Sequential(
            nn.Conv2d(128 + 128, 128, kernel_size=3, padding=1),  # Conv con concatenación de características (skip connection)
            nn.BatchNorm2d(128),  # Normalización por lotes
            nn.ReLU()  # Activación ReLU
        )

        # Segundo Up-sampling: de 128 a 64 canales
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = nn.Sequential(
            nn.Conv2d(64 + 64, 64, kernel_size=3, padding=1),  # Conv con concatenación (skip connection)
            nn.BatchNorm2d(64),  # Normalización por lotes
            nn.ReLU()  # Activación ReLU
        )

        # Capa de salida final
        self.output_layer = nn.Conv2d(64, 3, kernel_size=1)  # Conv para reducir a 3 canales de salida (RGB)
        self.output_activation = nn.Sigmoid()  # Activación sigmoid para limitar los valores entre 0 y 1

    def forward(self, x):
        # Encoder con Skip Connections
        enc1 = self.encoder1(x)  # Primer bloque del encoder: [B, 64, H, W]
        x = self.pool1(enc1)     # MaxPool para reducir la resolución: [B, 64, H/2, W/2]

        enc2 = self.encoder2(x)  # Segundo bloque del encoder: [B, 128, H/2, W/2]
        x = self.pool2(enc2)     # MaxPool para reducir la resolución: [B, 128, H/4, W/4]

        # Bottleneck (representación latente)
        x = self.bottleneck(x)   # [B, 256, H/4, W/4]

        # Decoder con Skip Connections
        x = self.upconv2(x)                           # Up-sampling: [B, 128, H/2, W/2]
        x = torch.cat((x, enc2), dim=1)               # Concatenar con salida del encoder2: [B, 128 + 128, H/2, W/2]
        x = self.decoder2(x)                          # Bloque de decodificación: [B, 128, H/2, W/2]

        x = self.upconv1(x)                           # Up-sampling: [B, 64, H, W]
        x = torch.cat((x, enc1), dim=1)               # Concatenar con salida del encoder1: [B, 64 + 64, H, W]
        x = self.decoder1(x)                          # Bloque de decodificación: [B, 64, H, W]

        # Capa de salida
        x = self.output_layer(x)                      # Conv para reducir a 3 canales (RGB): [B, 3, H, W]
        x = self.output_activation(x)                 # Aplicar activación sigmoid para obtener valores entre [0, 1]

        return x

    def training_step(self, batch, batch_idx):
        # Paso de entrenamiento
        x, _ = batch  # Obtener las imágenes (sin etiquetas)
        x_hat = self(x)  # Pasar por el autoencoder para obtener la reconstrucción
        loss = F.mse_loss(x_hat, x)  # Calcular la pérdida de error cuadrático medio
        mae_loss = F.l1_loss(x_hat, x)   # Calcular el error absoluto medio (MAE)

        # Registrar la pérdida y las métricas de entrenamiento
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)

        # Usar wandb para registrar métricas adicionales
        wandb.log({
            'train_loss': loss,
        })

        return loss

    def validation_step(self, batch, batch_idx):
        # Paso de validación
        x, _ = batch  # Obtener las imágenes (sin etiquetas)
        x_hat = self(x)  # Pasar por el autoencoder para obtener la reconstrucción
        loss = F.mse_loss(x_hat, x)  # Calcular la pérdida de error cuadrático medio
        mae_loss = F.l1_loss(x_hat, x)   # Calcular el error absoluto medio (MAE)

        # Registrar la pérdida y las métricas de validación
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)

        wandb.log({
            'val_loss': loss,
        })

        return loss

    def configure_optimizers(self):
        # Configuración del optimizador y el scheduler
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)  # Optimizador Adam
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
        # Retornar el optimizador y el scheduler
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
        }


### Clasificador A.

In [ ]:
class ClasificadorA(LightningModule):
    def __init__(self, num_classes=20, learning_rate=1e-4):
        super(ClasificadorA, self).__init__()
        self.learning_rate = learning_rate

        # Definir una arquitectura simple de clasificación
        self.network = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(128 * 32 * 32, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.network(x)

    def training_step(self, batch, batch_idx):
        # Obtener el lote de datos de entrada (imágenes) y las etiquetas verdaderas
        x, y = batch
        # Calcular la predicción del modelo
        y_hat = self(x)
        # Calcular la pérdida utilizando la entropía cruzada
        loss = F.cross_entropy(y_hat, y)
        # Calcular la precisión del lote
        acc = (y_hat.argmax(dim=1) == y).float().mean()

        # Registrar las métricas de entrenamiento
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_accuracy', acc, on_step=True, on_epoch=True, prog_bar=True)

        # Usar wandb para registrar métricas adicionales
        wandb.log({
            'train_loss': loss,
            'train_accuracy': acc
        })
        return loss


    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()

        # Registrar las métricas de validación
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_accuracy', acc, on_step=False, on_epoch=True, prog_bar=True)

        # Usar wandb para registrar métricas adicionales
        wandb.log({
            'val_loss': loss,
            'val_accuracy': acc
        })
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss"
        }

#### Entrenamientos para Clasificador A.

In [ ]:
def main():
  # Limpiar cualquier instancia previa de Hydra para evitar errores de inicialización duplicada
    if GlobalHydra.instance().is_initialized():
          GlobalHydra.instance().clear()

    initialize(config_path=".", version_base=None)
    cfg = compose(config_name="config")

    # Configuración del datamodule según los parámetros en cfg
    data_module = ButterfliesDataModule(
        data_dir=cfg.experiment.data_dir,
        batch_size=cfg.experiment.batch_size,
        labeled_ratio=cfg.experiment.labeled_ratio
    )
    data_module.setup()

    # 1. Entrenamiento del Autoencoder
    autoencoder = UNetAutoencoder(learning_rate=cfg.experiment.learning_rate_autoencoder)
    trainer_autoencoder = Trainer(
        max_epochs=cfg.experiment.max_epochs_autoencoder,
        callbacks=[EarlyStopping(monitor='train_loss', min_delta=0.01, patience=5, verbose=True, mode='min')],
        logger=pl.loggers.WandbLogger(project="butterfly-classifier", name="run_Autoencoder"),
        accelerator='auto'
    )
    trainer_autoencoder.fit(autoencoder, data_module.train_dataloader())

    # Guardar el autoencoder entrenado para su uso posterior
    torch.save(autoencoder.state_dict(), "autoencoder_trained.pth")
    wandb.finish()

    # 2. Entrenamiento del Clasificador A con los datos etiquetados
    clasificador_a = ClasificadorA(num_classes=20, learning_rate=cfg.experiment.learning_rate_classifier_a)
    trainer_clasificador_a = Trainer(
        max_epochs=cfg.experiment.max_epochs_classifier_a,
        callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.01, patience=10, verbose=True, mode='min')],
        logger=pl.loggers.WandbLogger(project="butterfly-classifier", name="run_A"),
        accelerator='auto'
    )
    trainer_clasificador_a.fit(clasificador_a, train_dataloaders=data_module.labeled_train_dataloader(), val_dataloaders=data_module.val_dataloader())
    wandb.finish()

if __name__ == "__main__":
    main()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
wandb: Currently logged in as: andrearaya1234 (andrearaya1234-tec). Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name              | Type            | Params | Mode 
---------------------------------------------------------------
0  | encoder1          | Sequential      | 1.9 K  | train
1  | pool1             | MaxPool2d       | 0      | train
2  | encoder2          | Sequential      | 74.1 K | train
3  | pool2             | MaxPool2d       | 0      | train
4  | bottleneck        | Sequential      | 295 K  | train
5  | upconv2           | ConvTranspose2d | 131 K  | train
6  | decoder2          | Sequential      | 295 K  | train
7  | upconv1           | ConvTranspose2d | 32.8 K | train
8  | decoder1          | Sequential      | 73.9 K | train
9  | output_layer      | Conv2d          | 195    | train
10 | output_activation | Sigmoid         | 0      | train
---------------------------------------------------------------
905 K     Trainable params
0         Non-trai

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved. New best score: 1.044
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.179 >= min_delta = 0.01. New best score: 0.865
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.026 >= min_delta = 0.01. New best score: 0.839
INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss improved by 0.012 >= min_delta = 0.01. New best score: 0.827
INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric train_loss did not improve in the last 5 records. Best score: 0.827. Signaling Trainer to stop.


epoch,▁▁▂▂▃▃▃▃▃▄▄▄▅▅▆▆▆▆▆▇▇▇██
train_loss:,▅▆▄█▃▁▄▅▁▆▄▅▄▅▃▅▅▂▅▁▄▄▄▃▂▃▆▄▄▃▂▅▄▃▄▅▂▃▄▃
train_loss_epoch,█▂▂▁▁▁▁▁▁▁
train_loss_step,▆▄▄▆▆▂▅▃▄▃▃▁▄█
trainer/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇███████
epoch,9
train_loss:,0.60228
train_loss_epoch,0.82183
train_loss_step,0.97138
trainer/global_step,739


...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.013 >= min_delta = 0.01. New best score: 0.091


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██
train_acc_epoch,▁▂▂▄▄▄▄▄▅▅▄▅▅▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇████████
train_acc_step,▅▄▄▄▁▂▆▄▃▃▅▅▆▄▆▆▅▅▄▆▃▇▇▆▇▆▇▆██▇█
train_accuracy,▁▅▃▄▃▄▅▅▂▄▄▄▆▄▅▇▅▅▅▆▆▅▇▆▇▅▅▅▇▅▇▄▇▅█▇▇▇█▇
train_loss,█▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_epoch,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▂▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇█████
val_acc,▁▁▁▁▂▁▂▁▂▂▁▂▃▄▅▄▆▆▇▆▇▇▇▇▇▆▆▇▇███████████
val_accuracy,▃▃▃▅▃▄▃▅▁▃▄▆▄▆▆▇▆▆▇▆▆▆▇█▇▅▇▇▇▆▇█▇▇█▇▇▆█▆
val_loss,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


### Clasificador B

In [ ]:
class ClasificadorB(LightningModule):
    def __init__(self, encoder, num_classes=20, learning_rate=1e-3, congelar_pesos=True):
        super().__init__()
        # Utilizar el encoder del autoencoder
        self.encoder = encoder
        self.learning_rate = learning_rate

        # Congelar los pesos del encoder si se especifica
        if congelar_pesos:
            for param in self.encoder.parameters():
                param.requires_grad = False

        # La salida del encoder es [batch_size, 3, 128, 128]
        # Necesitamos aplanarla a [batch_size, 49152]
        self.classifier = nn.Sequential(
            nn.Linear(3 * 128 * 128, 512),  # Primera capa lineal
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)  # Capa de salida
        )

    def forward(self, x):
        # Pasar las imágenes por el encoder
        with torch.no_grad():
            x = self.encoder(x)
        # Aplanar la salida del encoder
        x = x.view(x.size(0), -1)  # De [batch_size, 3, 128, 128] a [batch_size, 49152]
        # Pasar por el clasificador
        return self.classifier(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()

        self.log('train_loss', loss)
        self.log('train_acc', acc)

        wandb.log({'train_loss': loss})
        wandb.log({'train_accuracy': acc})
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()

        self.log('val_loss', loss)
        self.log('val_acc', acc)

        wandb.log({'val_loss': loss})
        wandb.log({'val_accuracy': acc})
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss"
        }

#### Entrenar Clasificador B.

In [ ]:
def main():
    if GlobalHydra.instance().is_initialized():
        GlobalHydra.instance().clear()

    # Inicializar Hydra manualmente
    initialize(config_path=".", version_base=None)

    # Obtener la configuración del archivo YAML
    cfg = compose(config_name="config")

    # Configuración del datamodule según los parámetros en cfg
    data_module = ButterfliesDataModule(
        data_dir=cfg.experiment.data_dir,
        batch_size=cfg.experiment.batch_size,
        labeled_ratio=cfg.experiment.labeled_ratio  # Agregar labeled_ratio al constructor
    )
    data_module.setup()

    # Carga el autoencoder ya creado por el Modelo A
    autoencoder = UNetAutoencoder()
    autoencoder.load_state_dict(torch.load("autoencoder_trained.pth"))

    # 1. Entrenamiento del Autoencoder
    # autoencoder = UNetAutoencoder(learning_rate=cfg.experiment.learning_rate_autoencoder)
    # trainer_autoencoder = Trainer(
        # max_epochs=cfg.experiment.max_epochs_autoencoder,
        # callbacks=[EarlyStopping(monitor='train_loss', min_delta=0.01, patience=5, verbose=True, mode='min')],
        # logger=pl.loggers.WandbLogger(project="butterfly-classifier", name="run_AutoencoderB"),
        # accelerator='auto'
    #)
    #trainer_autoencoder.fit(autoencoder, data_module.train_dataloader())
    #wandb.finish()

    # 2. Entrenar el Clasificador B1 (Pesos Congelados)
    clasificador_b1 = ClasificadorB(encoder=autoencoder, num_classes=20, learning_rate=cfg.experiment.learning_rate_classifier_b1, congelar_pesos=True)
    trainer_clasificador_b1 = Trainer(
        max_epochs=cfg.experiment.max_epochs_classifier_b1,
        callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.01, patience=10, verbose=True, mode='min')],
        logger=pl.loggers.WandbLogger(project="butterfly-classifier", name="run_B1"),
        accelerator='auto'
    )
    trainer_clasificador_b1.fit(clasificador_b1, train_dataloaders=data_module.labeled_train_dataloader(), val_dataloaders=data_module.val_dataloader())
    wandb.finish()

    # 3. Entrenar el Clasificador B2 (Pesos Sin Congelar)
    clasificador_b2 = ClasificadorB(encoder=autoencoder, num_classes=20, learning_rate=cfg.experiment.learning_rate_classifier_b2, congelar_pesos=False)
    trainer_clasificador_b2 = Trainer(
        max_epochs=cfg.experiment.max_epochs_classifier_b2,
        callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=True, mode='min')],
        logger=pl.loggers.WandbLogger(project="butterfly-classifier", name="run_B2"),
        accelerator='auto'
    )
    trainer_clasificador_b2.fit(clasificador_b2, train_dataloaders=data_module.labeled_train_dataloader(), val_dataloaders=data_module.val_dataloader())
    wandb.finish()

if __name__ == "__main__":
    main()

<ipython-input-11-10cc54f20abd>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  autoencoder.load_state_dict(torch.load("autoencoder_trained.pth"))
INFO:pytorch_lightning.u

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type            | Params | Mode 
-------------------------------------------------------
0 | encoder    | UNetAutoencoder | 905 K  | train
1 | classifier | Sequential      | 25.2 M | train
-------------------------------------------------------
25.2 M    Trainable params
905 K     Non-trainable params
26.1 M    Total params
104.327   Total estimated model params size (MB)
32        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (32) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_loss did not improve in the last 5 records. Best score: 0.112. Signaling Trainer to stop.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
train_acc,▁▂▅▆▃▂▂▆▇▅▄▇▆▇▄▆▇█▇▇▇█
train_accuracy,▁▅▄▂▆▅▅▅▃▅▆▄▆▆▆▇▄▆▆▇▇▇▆▇▆▆▆▆▇▇▇▆▆▆▇▆▆▆█▆
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇███
val_acc,▁▁▂▂▂▃▁▄▄▅▆▆▆▆▆▄▇▅▆▆▆▆▇▅▇▆███████▇▇
val_accuracy,▂▄▄▄▆▁▇▄▁▅▄▄▇▅▅▆▇▅▆▅▅█▅▇▇▆▅▆▇▇█▇▅▇▇▇▇▆▇▇
val_loss,▄▆█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,34
train_acc,1
train_accuracy,0.90909


## Experimento 2.

In [ ]:
class DenoisingClassifier(LightningModule):
    def __init__(self, num_classes=20, latent_dim=128):
        super().__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, latent_dim, kernel_size=3, padding=1),
            nn.ReLU(),
        )

        # Decoder for reconstruction
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(64, 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

        # Classifier head
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def add_noise(self, x, noise_prob=0.02):
        noise = torch.rand_like(x)
        # Salt noise
        salt_mask = noise < noise_prob
        x = torch.where(salt_mask, torch.ones_like(x), x)
        # Pepper noise
        pepper_mask = (noise > (1 - noise_prob))
        x = torch.where(pepper_mask, torch.zeros_like(x), x)
        return x

    def forward(self, x):
        features = self.encoder(x)
        reconstructed = self.decoder(features)
        logits = self.classifier(features)
        return reconstructed, logits, features

    def training_step(self, batch, batch_idx):
        x, y = batch
        x_noisy = self.add_noise(x)
        reconstructed, logits, _ = self(x_noisy)

        # Reconstruction loss
        recon_loss = F.mse_loss(reconstructed, x)
        # Classification loss
        class_loss = F.cross_entropy(logits, y)
        # Combined loss
        total_loss = recon_loss + class_loss

        self.log('train_recon_loss', recon_loss)
        self.log('train_class_loss', class_loss)
        self.log('train_total_loss', total_loss)

        return total_loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x_noisy = self.add_noise(x)
        reconstructed, logits, _ = self(x_noisy)

        recon_loss = F.mse_loss(reconstructed, x)
        class_loss = F.cross_entropy(logits, y)
        total_loss = recon_loss + class_loss

        # Calculate accuracy
        pred = torch.argmax(logits, dim=1)
        acc = (pred == y).float().mean()

        self.log('val_recon_loss', recon_loss)
        self.log('val_class_loss', class_loss)
        self.log('val_total_loss', total_loss)
        self.log('val_accuracy', acc)

        return total_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Reutilizar el DataModule del experimento 1
data_module.setup()

# Inicializar y entrenar el modelo
model = DenoisingClassifier()
trainer = Trainer(
    max_epochs=10,
    callbacks=[early_stopping],
    logger=pl.loggers.WandbLogger()
)
trainer.fit(model, datamodule=data_module)

# Análisis del espacio latente
def analyze_latent_space(model, data_module, n_clusters=20):
    model.eval()
    all_features = []
    all_labels = []

    # Extraer características del conjunto de datos no etiquetado
    with torch.no_grad():
        for batch in data_module.train_dataloader():  # Usando el conjunto no etiquetado
            x, _ = batch
            _, _, features = model(x)
            features = features.mean(dim=(2, 3))  # Promedio espacial de las características
            all_features.append(features)

    # Concatenar todas las características
    all_features = torch.cat(all_features).cpu().numpy()

    # Aplicar t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    features_2d = tsne.fit_transform(all_features)

    # Aplicar K-means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(all_features)

    # Visualización
    plt.figure(figsize=(12, 5))

    # t-SNE con clusters
    plt.subplot(1, 1, 1)
    scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1],
                         c=cluster_labels, cmap='tab20')
    plt.colorbar(scatter)
    plt.title('t-SNE visualization of latent space with K-means clusters')
    plt.xlabel('t-SNE dimension 1')
    plt.ylabel('t-SNE dimension 2')

    plt.tight_layout()
    plt.show()

    return features_2d, cluster_labels

# Ejecutar análisis
features_2d, cluster_labels = analyze_latent_space(model, data_module)

# Visualizar algunas reconstrucciones
def visualize_reconstructions(model, data_module, num_samples=5):
    model.eval()
    batch = next(iter(data_module.val_dataloader()))
    images, labels = batch
    images = images[:num_samples]

    with torch.no_grad():
        noisy_images = model.add_noise(images)
        reconstructed, _, _ = model(noisy_images)

    # Visualización
    fig, axes = plt.subplots(3, num_samples, figsize=(15, 6))

    for i in range(num_samples):
        # Original
        axes[0, i].imshow(images[i].permute(1, 2, 0).cpu())
        axes[0, i].axis('off')
        if i == 0: axes[0, i].set_title('Original')

        # Noisy
        axes[1, i].imshow(noisy_images[i].permute(1, 2, 0).cpu())
        axes[1, i].axis('off')
        if i == 0: axes[1, i].set_title('Noisy')

        # Reconstructed
        axes[2, i].imshow(reconstructed[i].permute(1, 2, 0).cpu())
        axes[2, i].axis('off')
        if i == 0: axes[2, i].set_title('Reconstructed')

    plt.tight_layout()
    plt.show()

# Visualizar reconstrucciones
visualize_reconstructions(model, data_module)
